In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from pathlib import Path
import torch
from sentence_transformers import models, losses, datasets
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import pandas as pd
from tqdm import tqdm
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [6]:
X_train_vec = np.load("wmt_data/X_train.npy")
X_eval_vec = np.load("wmt_data/X_eval.npy")
X_test_vec = np.load("wmt_data/X_test.npy")

dataset = DatasetDict.load_from_disk("wmt_data/full_dataset")

In [121]:
clf = SVR()
n_samples = 5000
clf.fit(X_train_vec[:n_samples], dataset['train']['label'][:n_samples])

SVR()

In [122]:
labels_pred = clf.predict(X_train_vec[:n_samples])
print("MSE train:", mean_squared_error(dataset['train']['label'][:n_samples], labels_pred))

labels_pred_eval = clf.predict(X_eval_vec)
print("MSE eval:", mean_squared_error(dataset['eval']['label'], labels_pred_eval))

MSE train: 0.2628466066157682
MSE eval: 0.2963647795068554


In [15]:
def svr_metric(out, ref):
    vec_out = sentence_model.encode(out)
    vec_ref = sentence_model.encode(ref)
    return clf.predict([np.concatenate([vec_ref, vec_out])])

def nn_metric(out, ref):
    return model(out, ref)

scorer = nn_metric

print(scorer("It's rather hot in here.", "It's rather hot in here."))
print(scorer("It's rather hot in here.", "It's rather warm in here."))
print(scorer("It's rather hot in here.", "It's rather not hot in here."))
print(scorer("Luckily, people are interested in borrowing skis.", "Fortunately, there is interest in ski rental."))

tensor([0.1112], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.1507], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0709], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0452], device='cuda:0', grad_fn=<AddBackward0>)


In [124]:
import pickle
with open("wmt_data_bleurt/svr_5k.pkl", 'wb') as f:
    pickle.dump(clf, f)

with open("wmt_data/svr_10k.pkl", 'rb') as f:
    clf = pickle.load(f)

In [35]:
#import mtMetricsEval.mt_metrics_eval as mte
#import mtMetricsEval.mt_metrics_eval.data
from mtMetricsEval.mt_metrics_eval import data

evs = data.EvalSet('wmt20', 'en-de')
scores = {level: {} for level in ['sys', 'doc', 'seg']}
ref = evs.all_refs[evs.std_ref]
for s, out in evs.sys_outputs.items():
  scores['sys'][s] = [svr_metric(out, ref)]
  scores['doc'][s] = [svr_metric(out[b:e], ref[b:e]) for b, e in evs.docs.values()]
  scores['seg'][s] = [svr_metric([o], [r]) for o, r in zip(out, ref)]

# Official WMT correlations.
for level in 'sys', 'doc', 'seg':
  gold_scores = evs.Scores(level, evs.StdHumanScoreName(level))
  sys_names = set(gold_scores) - evs.human_sys_names
  corr = evs.Correlation(gold_scores, scores[level], sys_names)
  print(f'{level}: Pearson={corr.Pearson()[0]:f}, '
        f'Kendall-like={corr.KendallLike()[0]:f}')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 4040: character maps to <undefined>

In [34]:
import sys
sys.path.append("mtMetricsEval")

## NLI tutorial

In [9]:
import math
from sentence_transformers import models, losses, datasets
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datasets import DatasetDict, Dataset
from tqdm import tqdm
from pathlib import Path
import torch
from datetime import datetime
import sys
import os
import gzip
import csv
import random

In [4]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [5]:
timestamp = "2023-02-17_15-02-13"
project_base_path = Path("Guided Research WS22")
negation_dataset = project_base_path / "data/negation_dataset_labeled.tsv"

base_model = "sentence-transformers/all-mpnet-base-v2"
output_model_name = f"{base_model.split('/')[1]}-negation"  # TODO.
model_save_path = str(project_base_path / f"finetuned-models/{timestamp}/{output_model_name}")

model_name = model_save_path
train_batch_size = 2          #The larger you select this, the better the results (usually). But it requires more GPU memory
max_seq_length = 75
num_epochs = 1
data_save_path = "wmt_data_augmented"

model_save_path = 'output/nli_model_regression'

In [6]:
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

dataset = DatasetDict.load_from_disk(data_save_path)
logging.info("Loaded models and data")

2023-03-31 14:17:51 - Use pytorch device: cuda
2023-03-31 14:17:51 - Loaded models and data


In [7]:
def dataset_to_InputExample(dataset: Dataset, ref_column: str, cand_column: str, lab_column:str) -> list[InputExample] :
    return [
        InputExample(i, texts=[x[ref_column], x[cand_column]], label=x[lab_column])
        for i, x in tqdm(enumerate(dataset), total=len(dataset))
    ]

train_samples = dataset_to_InputExample(dataset['train'], 'reference', 'candidate', 'score')[:1000]
dev_samples = dataset_to_InputExample(dataset['eval'], 'reference', 'candidate', 'score')
test_samples = dataset_to_InputExample(dataset['test'], 'reference', 'candidate', 'score')

100%|██████████| 48408/48408 [00:03<00:00, 13732.56it/s]


In [16]:
logging.info("Train samples: {}".format(len(train_samples)))
# Special data loader that avoid duplicates within a batch
train_dataloader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=train_batch_size)
# Our training loss
#train_loss = losses.MultipleNegativesRankingLoss(model)
train_loss = losses.SoftmaxLoss(model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=1, loss_fct=torch.nn.MSELoss())

2023-03-31 14:21:45 - Train samples: 1000
2023-03-31 14:21:45 - Softmax loss: #Vectors concatenated: 3


In [17]:
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2023-03-31 14:21:50 - Warmup-steps: 50


In [18]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=int(len(train_dataloader)*0.5),
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          use_amp=True          #Set to True, if your GPU supports FP16 operations
          )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/500 [00:00<?, ?it/s]

2023-03-31 14:23:09 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 250 steps:
2023-03-31 14:37:58 - Cosine-Similarity :	Pearson: 0.4656	Spearman: 0.5695
2023-03-31 14:37:58 - Manhattan-Distance:	Pearson: 0.5200	Spearman: 0.5620
2023-03-31 14:37:58 - Euclidean-Distance:	Pearson: 0.5207	Spearman: 0.5626
2023-03-31 14:37:58 - Dot-Product-Similarity:	Pearson: 0.4065	Spearman: 0.4219
2023-03-31 14:37:58 - Save model to output/nli_model_regression


d:\users\miriam anschütz\documents\workspaces\pycharm projects\negation_aware_evaluation\venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


2023-03-31 14:39:17 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 500 steps:
2023-03-31 14:54:12 - Cosine-Similarity :	Pearson: 0.4735	Spearman: 0.5747
2023-03-31 14:54:12 - Manhattan-Distance:	Pearson: 0.5165	Spearman: 0.5599
2023-03-31 14:54:12 - Euclidean-Distance:	Pearson: 0.5172	Spearman: 0.5605
2023-03-31 14:54:12 - Dot-Product-Similarity:	Pearson: 0.3775	Spearman: 0.3937
2023-03-31 14:54:12 - Save model to output/nli_model_regression
2023-03-31 14:54:13 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2023-03-31 15:09:08 - Cosine-Similarity :	Pearson: 0.4735	Spearman: 0.5747
2023-03-31 15:09:08 - Manhattan-Distance:	Pearson: 0.5165	Spearman: 0.5599
2023-03-31 15:09:08 - Euclidean-Distance:	Pearson: 0.5172	Spearman: 0.5605
2023-03-31 15:09:08 - Dot-Product-Similarity:	Pearson: 0.3775	Spearman: 0.3937


In [18]:
model.encode(train_samples)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

TypeError: 'InputExample' object is not subscriptable

In [14]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
test_evaluator(model, output_path=model_save_path)

2023-03-31 11:20:28 - Load pretrained SentenceTransformer: output/nli_model
2023-03-31 11:20:32 - Use pytorch device: cuda
2023-03-31 11:20:32 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2023-03-31 11:36:34 - Cosine-Similarity :	Pearson: 0.4482	Spearman: 0.5639
2023-03-31 11:36:34 - Manhattan-Distance:	Pearson: 0.5110	Spearman: 0.5602
2023-03-31 11:36:34 - Euclidean-Distance:	Pearson: 0.5111	Spearman: 0.5602
2023-03-31 11:36:34 - Dot-Product-Similarity:	Pearson: 0.3696	Spearman: 0.3909


0.5638637912636975

In [15]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 75, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)